In [47]:
from spacy.tokens import Span
from spacy.matcher import Matcher
from tqdm import tqdm
import matplotlib.pyplot as plt
import networkx as nx
import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy

import neuralcoref
import logging
logging.basicConfig(level=logging.INFO)


import en_core_web_lg
#nlp = en_core_web_lg.load()
nlp = spacy.load('en_core_web_lg')
neuralcoref.add_to_pipe(nlp)
spacy.prefer_gpu()

False

In [2]:
def get_entities(sent):
    # print('test')
    # chunk 1
    ent1 = ""
    ent2 = ""

    prv_tok_dep = ""    # dependency tag of previous token in the sentence
    prv_tok_text = ""   # previous token in the sentence

    prefix = ""
    modifier = ""

    #############################################################
    print(nlp(sent))
    for tok in nlp(sent):
        # chunk 2
        # if token is a punctuation mark then move on to the next token
        print(tok)
        if tok.dep_ != "punct":
                # check: token is a compound word or not
            if tok.dep_ == "compound":
                prefix = tok.text
                # if the previous word was also a 'compound' then add the current word to it
                if prv_tok_dep == "compound":
                    prefix = prv_tok_text + " " + tok.text

            # check: token is a modifier or not
            if tok.dep_.endswith("mod") == True:
                modifier = tok.text
                # if the previous word was also a 'compound' then add the current word to it
                if prv_tok_dep == "compound":
                    modifier = prv_tok_text + " " + tok.text

            # chunk 3
            if tok.dep_.find("subj") == True:
                ent1 = modifier + " " + prefix + " " + tok.text
                prefix = ""
                modifier = ""
                prv_tok_dep = ""
                prv_tok_text = ""

            # chunk 4
            if tok.dep_.find("obj") == True:
                ent2 = modifier + " " + prefix + " " + tok.text

            # chunk 5
            # update variables
            prv_tok_dep = tok.dep_
            prv_tok_text = tok.text
        #############################################################

        return [ent1.strip(), ent2.strip()]


# print(get_entities("he is a good boy"))


In [3]:



from spacy.lang.en import English
from spacy.pipeline import EntityRuler

# ruler = EntityRuler(nlp)
# # patterns = [{"label": "ORG", "pattern": "Apple"},
# #             {"label": "GPE", "pattern": [{"LOWER": "san"}, {"LOWER": "francisco"}]}]
# patterns = [{'pattern': 'enfj', 'label': 'nsubj'}]
# ruler.add_patterns(patterns)
# nlp.add_pipe(ruler)

# doc = nlp("Apple is opening its first big office in San Francisco.")
# print([(ent.text, ent.label_) for ent in doc.ents])





def entity_pairs(text, coref=True):
    text = re.sub(r'\n+', '.', text)  # replace multiple newlines with period
    text = re.sub(r'\[\d+\]', ' ', text)  # remove reference numbers
    text = nlp(text)
    print(text)
    if coref:
        text = nlp(text._.coref_resolved)  # resolve coreference clusters
    sentences = [sent.string.strip() for sent in text.sents]  # split text into sentences
    ent_pairs = list()
    for sent in sentences:
        pattern = [{},
           {'LEMMA': 'tool', 'POS': 'NOUN'},
           {}]
        sent = nlp(sent)
        spans = list(sent.ents) + list(sent.noun_chunks)  # collect nodes
        spans = spacy.util.filter_spans(spans)
        with sent.retokenize() as retokenizer:
            [retokenizer.merge(span) for span in spans]
        dep = [token.dep_ for token in sent]
        if (dep.count('obj')+dep.count('dobj'))==1 \
                and (dep.count('subj')+dep.count('nsubj'))==1:
            for token in sent:
                if token.dep_ in ('obj', 'dobj'):  # identify object nodes
                    subject = [w for w in token.head.lefts if w.dep_
                               in ('subj', 'nsubj')]  # identify subject nodes
                    if subject:
                        subject = subject[0]
                        # identify relationship by root dependency
                        relation = [w for w in token.ancestors if w.dep_ == 'ROOT']  
                        if relation:
                            relation = relation[0]
                            # add adposition or particle to relationship
                            if relation.nbor(1).pos_ in ('ADP', 'PART'):  
                                relation = ' '.join((str(relation),
                                        str(relation.nbor(1))))
                        else:
                            relation = 'unknown'
                        subject, subject_type = refine_ent(subject, sent)
                        token, object_type = refine_ent(token, sent)
                        ent_pairs.append([str(subject), str(relation), str(token),
                                str(subject_type), str(object_type)])
    filtered_ent_pairs = [sublist for sublist in ent_pairs
                          if not any(str(x) == '' for x in sublist)]
    pairs = pd.DataFrame(filtered_ent_pairs, columns=['subject',
                         'relation', 'object', 'subject_type',
                         'object_type'])
    print('Entity pairs extracted:', str(len(filtered_ent_pairs)))
    return pairs



In [4]:
def refine_ent(ent, sent):
    unwanted_tokens = (
        'PRON',  # pronouns
        'PART',  # particle
        'DET',  # determiner
        'SCONJ',  # subordinating conjunction
        'PUNCT',  # punctuation
        'SYM',  # symbol
        'X',  # other
        )
    ent_type = ent.ent_type_  # get entity type
    if ent_type == '':
        ent_type = 'NOUN_CHUNK'
        ent = ' '.join(str(t.text) for t in
                nlp(str(ent)) if t.pos_
                not in unwanted_tokens and t.is_stop == False)
    elif ent_type in ('NOMINAL', 'CARDINAL', 'ORDINAL') and str(ent).find(' ') == -1:
        t = ''
        for i in range(len(sent) - ent.i):
            if ent.nbor(i).pos_ not in ('VERB', 'PUNCT'):
                t += ' ' + str(ent.nbor(i))
            else:
                ent = t.strip()
                break
    return ent, ent_type


import wikipediaapi
import pandas as pd

def wiki_page(page_name):
    wiki_api = wikipediaapi.Wikipedia(language='en',
                                      extract_format=wikipediaapi.ExtractFormat.WIKI)
    page_name = wiki_api.page(page_name)
    if not page_name.exists():
        print('page does not exist')
        return
    page_data = {'page': page_name, 'text': page_name.text, 'link': page_name.fullurl,
                 'categories': [[y[9:] for y in list(page_name.categories.keys())]]}
    page_data_df = pd.DataFrame(page_data)
    return page_data_df

# wiki_data = wiki_page('dj')

# print(wiki_data.loc[0,'text'])


In [104]:
enfj = """ The ENFJ Preferences are Extraversion , Intuition , Feeling and Judging . These determine the ENFJ Personality Type . 
Extraverted (E) Extraversion is characterized by a preference to focus on the world outside the self. ENFJs are energized by social gatherings, parties 
and group activities. Extraverts are usually enthusiastic, gregarious and animated. Their communication style is verbal and assertive. As Extraverts, ENFJ often 
need to talk. They enjoy the limelight. Feeling (F) As Feeling people, ENFJs are subjective. They make decisions based on their principles and values. They are ruled 
by their heart instead of their head. ENFJs judge situations and others based on their feelings and extenuating circumstances. They seek to please others and want to 
be appreciated. They value harmony and empathy. Intuitive (N) People with Intuition live in the future. They are immersed in the world of possibilities. They process 
information through patterns and impressions. As Intuitives, ENFJ value inspiration and imagination. They gather knowledge by reading between the lines. Their abstract 
nature attracts them toward deep ideas, concepts and metaphors. Judging (J) As Judging people, ENFJs think sequentially. They value order and organization. Their lives 
are scheduled and structured. ENFJs seek closure and enjoy completing tasks. They take deadlines seriously. They work then they play. The Judging preference does not mean
 judgmental . Judging refers to how a day-to-day activities at dealt with. ENFJ is cool."""

# print(entity_pairs(enfj))



from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)
matched_sents = []

def collect_sents(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    span = doc[start:end]  # Matched span
    sent = span.sent  # Sentence containing matched span
    # Append mock entity for match in displaCy style to matched_sents
    # get the match span by ofsetting the start and end of the span with the
    # start and end of the sentence in the doc
    match_ents = [{
        "start": span.start_char - sent.start_char,
        "end": span.end_char - sent.start_char,
        "label": "MATCH",
    }]
    matched_sents.append({"text": sent.text, "ents": match_ents})

pattern = [{"LOWER": "enfj"}, {"LEMMA": "be"}, {"POS": "ADV", "OP": "*"},
           {"POS": "ADJ"}]
matcher.add("enfj", collect_sents, pattern)  # add pattern
doc = nlp(enfj)
matches = matcher(doc)

print(matches)
displacy.render(matched_sents, style="ent", manual=True)


# d = {'text': ['He is a good boy. Sometimes he does no do his homework']}
# df = pd.DataFrame(data=d)
# print(entity_pairs(wiki_data.loc[0,'text']))

[(9858085948341774798, 281, 284)]


In [9]:
import pprint
pp = pprint.PrettyPrinter(
    indent=2, width=80, depth=None, stream=None, compact=False)

In [10]:

from allennlp.predictors.predictor import Predictor
import allennlp_models.coref
import allennlp_models.structured_prediction
corefPredictor = Predictor.from_path(
    "https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz", cuda_device=0)
pp.pprint(corefPredictor.predict(document=enfj))

INFO:allennlp.common.file_utils:checking cache for https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz at C:\Users\djway\.allennlp\cache\0f6b052811b20b13280e609a96efe71ebc636b9c823a5c906ba24459e6e68af9.c1dab61d84cc7c3f7d6751c260040607cb7023a002778ba8f9b9d196b6539174
INFO:allennlp.common.file_utils:waiting to acquire lock on C:\Users\djway\.allennlp\cache\0f6b052811b20b13280e609a96efe71ebc636b9c823a5c906ba24459e6e68af9.c1dab61d84cc7c3f7d6751c260040607cb7023a002778ba8f9b9d196b6539174
INFO:filelock:Lock 2311697803656 acquired on C:\Users\djway\.allennlp\cache\0f6b052811b20b13280e609a96efe71ebc636b9c823a5c906ba24459e6e68af9.c1dab61d84cc7c3f7d6751c260040607cb7023a002778ba8f9b9d196b6539174.lock
INFO:allennlp.common.file_utils:cache of https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz is up-to-date
INFO:filelock:Lock 2311697803656 released on C:\Users\djway\.allennlp\cache\0f6b052811b20b13280e609a96efe71ebc636

INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/SpanBERT/spanbert-large-cased/vocab.txt from cache at C:\Users\djway/.cache\torch\transformers\3c791b35663b47a1c79ed04d06cd628f11a0e1ac5248c736e3e63437dd140820.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/SpanBERT/spanbert-large-cased/added_tokens.json from cache at None
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/SpanBERT/spanbert-large-cased/special_tokens_map.json from cache at None
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/SpanBERT/spanbert-large-cased/tokenizer_config.json from cache at None
INFO:allennlp.common.params:model.context_layer.type = pass_through
INFO:allennlp.common.params:model.context_layer.input_dim = 1024
INFO:allennlp.common.pa

INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.0.output.LayerNorm.bias
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.0.output.LayerNorm.weight
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.0.output.dense.bias
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.0.output.dense.weight
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.1.attention.output.LayerNorm.bias
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.1.attention.output.LayerNorm.weight
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._match

INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.12.attention.output.dense.bias
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.12.attention.output.dense.weight
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.12.attention.self.key.bias
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.12.attention.self.key.weight
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.12.attention.self.query.bias
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.12.attention.self.query.weight
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embe

INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.15.attention.self.query.weight
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.15.attention.self.value.bias
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.15.attention.self.value.weight
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.15.intermediate.dense.bias
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.15.intermediate.dense.weight
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.15.output.LayerNorm.bias
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tok

...

INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.6.output.LayerNorm.weight
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.6.output.dense.bias
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.6.output.dense.weight
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.7.attention.output.LayerNorm.bias
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.7.attention.output.LayerNorm.weight
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens._matched_embedder.transformer_model.encoder.layer.7.attention.output.dense.bias
INFO:allennlp.nn.initializers:   _text_field_embedder.token_embedder_tokens.

INFO:root:Loading a model trained before embedding extension was implemented; pass an explicit vocab namespace if you want to extend the vocabulary.
INFO:allennlp.common.params:validation_dataset_reader.type = coref
INFO:allennlp.common.params:validation_dataset_reader.lazy = False
INFO:allennlp.common.params:validation_dataset_reader.cache_directory = None
INFO:allennlp.common.params:validation_dataset_reader.max_instances = None
INFO:allennlp.common.params:validation_dataset_reader.manual_distributed_sharding = False
INFO:allennlp.common.params:validation_dataset_reader.manual_multi_process_sharding = False
INFO:allennlp.common.params:validation_dataset_reader.max_span_width = 30
INFO:allennlp.common.params:validation_dataset_reader.token_indexers.tokens.type = pretrained_transformer_mismatched
INFO:allennlp.common.params:validation_dataset_reader.token_indexers.tokens.token_min_padding_length = 0
INFO:allennlp.common.params:validation_dataset_reader.token_indexers.tokens.model_name 

{ 'antecedent_indices': [ [ 0,
                            1,
                            2,
                            3,
                            4,
                            5,
                            6,
                            7,
                            8,
                            9,
                            10,
                            11,
                            12,
                            13,
                            14,
                            15,
                            16,
                            17,
                            18,
                            19,
                            20,
                            21,
                            22,
                            23,
                            24,
                            25,
                            26,
                            27,
                            28,
                            29,
                            30,
                  

                            13,
                            14,
                            15,
                            16,
                            17,
                            18,
                            19,
                            20,
                            21,
                            22,
                            23,
                            24,
                            25,
                            26,
                            27,
                            28,
                            29,
                            30,
                            31,
                            32,
                            33,
                            34,
                            35,
                            36,
                            37,
                            38,
                            39,
                            40,
                            41,
                            42,
                            43,
        

                            29,
                            32,
                            36,
                            38,
                            39,
                            40,
                            41,
                            42,
                            44,
                            45,
                            46,
                            47,
                            48,
                            49,
                            50,
                            55,
                            56,
                            57,
                            59,
                            60,
                            61,
                            66,
                            68,
                            70,
                            71,
                            74,
                            78,
                            83,
                            84,
                            85,
                            86,
        

In [105]:
from nltk.tokenize import sent_tokenize
sentences = sent_tokenize(enfj)


In [14]:
openPredictor = Predictor.from_path(
    "https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz", cuda_device=0)
openPredictor.predict(sentence=sentences[0])

INFO:allennlp.common.file_utils:checking cache for https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz at C:\Users\djway\.allennlp\cache\60314a853eb0aaa774d176d878c62469d49872feb4f2bfd071a75c77f6d76707.1b91cc27e347f2df04ce771a304bee2b70a2c487626b67e277d44c593b868c25
INFO:allennlp.common.file_utils:waiting to acquire lock on C:\Users\djway\.allennlp\cache\60314a853eb0aaa774d176d878c62469d49872feb4f2bfd071a75c77f6d76707.1b91cc27e347f2df04ce771a304bee2b70a2c487626b67e277d44c593b868c25
INFO:filelock:Lock 2311697747528 acquired on C:\Users\djway\.allennlp\cache\60314a853eb0aaa774d176d878c62469d49872feb4f2bfd071a75c77f6d76707.1b91cc27e347f2df04ce771a304bee2b70a2c487626b67e277d44c593b868c25.lock
INFO:allennlp.common.file_utils:cache of https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz is up-to-date
INFO:filelock:Lock 2311697747528 released on C:\Users\djway\.allennlp\cache\60314a853eb0aaa774d176d878c62469d49872feb4f2bfd071a75

INFO:allennlp.common.params:dataset_reader.lazy = False
INFO:allennlp.common.params:dataset_reader.cache_directory = None
INFO:allennlp.common.params:dataset_reader.max_instances = None
INFO:allennlp.common.params:dataset_reader.manual_distributed_sharding = False
INFO:allennlp.common.params:dataset_reader.manual_multi_process_sharding = False
INFO:allennlp.common.params:dataset_reader.token_indexers = None
INFO:allennlp.common.params:dataset_reader.domain_identifier = None
INFO:allennlp.common.params:dataset_reader.bert_model_name = None


{'verbs': [{'verb': 'are',
   'description': '[ARG1: The ENFJ Preferences] [V: are] [ARG2: Extraversion] , Intuition , Feeling and Judging .These determine the ENFJ Personality Type .',
   'tags': ['B-ARG1',
    'I-ARG1',
    'I-ARG1',
    'B-V',
    'B-ARG2',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O']},
  {'verb': 'determine',
   'description': 'The ENFJ Preferences are Extraversion , [ARG0: Intuition , Feeling and Judging .These] [V: determine] [ARG1: the ENFJ Personality Type] .',
   'tags': ['O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'B-ARG0',
    'I-ARG0',
    'I-ARG0',
    'I-ARG0',
    'I-ARG0',
    'I-ARG0',
    'B-V',
    'B-ARG1',
    'I-ARG1',
    'I-ARG1',
    'I-ARG1',
    'O']}],
 'words': ['The',
  'ENFJ',
  'Preferences',
  'are',
  'Extraversion',
  ',',
  'Intuition',
  ',',
  'Feeling',
  'and',
  'Judging',
  '.These',
  'determine',
  'the',
  'ENFJ',
  'Personality',
  'Type',
  '

In [97]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.structured_prediction
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/biaffine-dependency-parser-ptb-2020.04.06.tar.gz")
predictor.predict(
  sentence=sentences[2]
)

INFO:allennlp.common.file_utils:checking cache for https://storage.googleapis.com/allennlp-public-models/biaffine-dependency-parser-ptb-2020.04.06.tar.gz at C:\Users\djway\.allennlp\cache\027f97737d5c3960a71add5e3e6f1450bc115e7a8c61d650a417c512cf4c7c4d.1ef6de261fafbc3ca58ddb3980b081d5726a41acc4533328b7bcd78175f7fc6e
INFO:allennlp.common.file_utils:waiting to acquire lock on C:\Users\djway\.allennlp\cache\027f97737d5c3960a71add5e3e6f1450bc115e7a8c61d650a417c512cf4c7c4d.1ef6de261fafbc3ca58ddb3980b081d5726a41acc4533328b7bcd78175f7fc6e
INFO:filelock:Lock 2311450043144 acquired on C:\Users\djway\.allennlp\cache\027f97737d5c3960a71add5e3e6f1450bc115e7a8c61d650a417c512cf4c7c4d.1ef6de261fafbc3ca58ddb3980b081d5726a41acc4533328b7bcd78175f7fc6e.lock
INFO:allennlp.common.file_utils:cache of https://storage.googleapis.com/allennlp-public-models/biaffine-dependency-parser-ptb-2020.04.06.tar.gz is up-to-date
INFO:filelock:Lock 2311450043144 released on C:\Users\djway\.allennlp\cache\027f97737d5c3960a

INFO:allennlp.nn.initializers:   encoder._module.backward_layer_0.cell.input_linearity.weight
INFO:allennlp.nn.initializers:   encoder._module.backward_layer_0.cell.state_linearity.bias
INFO:allennlp.nn.initializers:   encoder._module.backward_layer_0.cell.state_linearity.weight
INFO:allennlp.nn.initializers:   encoder._module.backward_layer_1.cell.input_linearity.weight
INFO:allennlp.nn.initializers:   encoder._module.backward_layer_1.cell.state_linearity.bias
INFO:allennlp.nn.initializers:   encoder._module.backward_layer_1.cell.state_linearity.weight
INFO:allennlp.nn.initializers:   encoder._module.backward_layer_2.cell.input_linearity.weight
INFO:allennlp.nn.initializers:   encoder._module.backward_layer_2.cell.state_linearity.bias
INFO:allennlp.nn.initializers:   encoder._module.backward_layer_2.cell.state_linearity.weight
INFO:allennlp.nn.initializers:   encoder._module.forward_layer_0.cell.input_linearity.weight
INFO:allennlp.nn.initializers:   encoder._module.forward_layer_0.ce

{'arc_loss': 0.5574563145637512,
 'tag_loss': 0.36357808113098145,
 'loss': 0.9210343956947327,
 'words': ['ENFJs',
  'are',
  'energized',
  'by',
  'social',
  'gatherings',
  ',',
  'parties',
  'and',
  'group',
  'activities',
  '.'],
 'pos': ['NOUN',
  'VERB',
  'VERB',
  'ADP',
  'ADJ',
  'NOUN',
  'PUNCT',
  'NOUN',
  'CCONJ',
  'NOUN',
  'NOUN',
  'PUNCT'],
 'predicted_dependencies': ['nsubj',
  'auxpass',
  'root',
  'prep',
  'amod',
  'pobj',
  'dep',
  'dep',
  'cc',
  'nn',
  'conj',
  'punct'],
 'predicted_heads': [2, 3, 0, 3, 6, 4, 8, 11, 8, 11, 6, 3],
 'hierplane_tree': {'text': 'ENFJs are energized by social gatherings , parties and group activities .',
  'root': {'word': 'energized',
   'nodeType': 'root',
   'attributes': ['VERB'],
   'link': 'root',
   'spans': [{'start': 10, 'end': 20}],
   'children': [{'word': 'are',
     'nodeType': 'auxpass',
     'attributes': ['VERB'],
     'link': 'auxpass',
     'spans': [{'start': 6, 'end': 10}],
     'children': [{'word'

In [106]:

#doc = nlp("Alex Smith worked at Acme Corp Inc.")
#print([(ent.text, ent.label_) for ent in doc.ents])
#print(doc)
#docs = list(nlp.pipe(sentences))
#print(docs)
#for doc in docs:
#    print([(ent.text, ent.label_) for ent in doc.ents])
    
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
import en_core_web_lg
#nlp = en_core_web_lg.load()
nlp = spacy.load('en_core_web_lg')

#nlp = English()
ruler = EntityRuler(nlp, overwrite_ents=True)
#patterns = [{"label": "PERSON", "pattern": [{"LOWER": "enfj"}]}]
patterns = [{"label": "PERSON", "pattern": [{"LOWER": {"REGEX": "[e|i]+[n|s]+[t|f]+[j+p]"}},{'LEMMA': 'be', 'OP': '?'}]}]
ruler.add_patterns(patterns)
nlp.add_pipe(ruler)

docs = list(nlp.pipe(sentences))
displacy.render(docs[0])

#for doc in docs:
    #for token in doc:
     #   print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_)
    #displacy.render(docs[0])
    #displacy.render(doc, style="ent")
    #for token in doc:
     #   print(token.text, token.dep_, token.head.text, token.head.pos_,
      #      [child for child in token.children])
    
    #doc.retokenize()
    #print([(ent.text, ent.label_) for ent in doc.ents])
    #print(doc.user_hooks)
    #chunks = list(doc.noun_chunks)
    #print([(ent.text, ent.label_) for ent in doc.noun_chunks])
    #print(chunks)
    #print(doc.text)
    #chunks = list(doc.noun_chunks)
    #for chunk in chunks:
     #   print(chunk)
    #for ent in doc.ents:
    #    print([(ent.text, ent.label_)])
    #for sent in doc._tokens :
     #   print([(sent.text, sent.label_)])

In [62]:
from spacy.lang.en import English
from spacy.pipeline import EntityRuler

#import re
#regex = re.compile('([e|i]+[n|s]+[t|f]+[j+p])')
nlp = English()
ruler = EntityRuler(nlp)
patterns = [{"label": "PERSON", 
             "pattern": [
                 {"LOWER": 
                  {"REGEX": "[e|i]+[n|s]+[t|f]+[j+p]"}
                 },
                 {'LEMMA': 'be', 'OP': '?'},
           {'LEMMA': 'be'}
             ]
            }]
#pattern = [{"TEXT": {"REGEX": "[e|i]+[n|s]+[t|f]+[j+p]"}}]
ruler.add_patterns(patterns)
nlp.add_pipe(ruler)

doc = nlp(sentences[0])
print([(ent.text, ent.label_) for ent in doc.ents])

[]


In [88]:
pattern = [{"TEXT": {"REGEX": "[e|i]+[n|s]+[t|f]+[j+p]"}}]
for doc in docs:
    print(doc.to_json())
    #for sent in doc.sents:
        #print(doc.to_json())
        #print(sent.text)

{'text': ' The ENFJ Preferences are Extraversion , Intuition , Feeling and Judging .These determine the ENFJ Personality Type .', 'ents': [{'start': 5, 'end': 9, 'label': 'PERSON'}, {'start': 94, 'end': 98, 'label': 'PERSON'}], 'sents': [{'start': 0, 'end': 117}], 'tokens': [{'id': 0, 'start': 0, 'end': 1, 'pos': 'SPACE', 'tag': '_SP', 'dep': '', 'head': 1}, {'id': 1, 'start': 1, 'end': 4, 'pos': 'DET', 'tag': 'DT', 'dep': 'det', 'head': 2}, {'id': 2, 'start': 5, 'end': 9, 'pos': 'PROPN', 'tag': 'NNP', 'dep': 'compound', 'head': 3}, {'id': 3, 'start': 10, 'end': 21, 'pos': 'NOUN', 'tag': 'NNS', 'dep': 'nsubj', 'head': 4}, {'id': 4, 'start': 22, 'end': 25, 'pos': 'VERB', 'tag': 'VBP', 'dep': 'ROOT', 'head': 4}, {'id': 5, 'start': 26, 'end': 38, 'pos': 'PROPN', 'tag': 'NNP', 'dep': 'attr', 'head': 4}, {'id': 6, 'start': 39, 'end': 40, 'pos': 'PUNCT', 'tag': ',', 'dep': 'punct', 'head': 5}, {'id': 7, 'start': 41, 'end': 50, 'pos': 'PROPN', 'tag': 'NNP', 'dep': 'conj', 'head': 5}, {'id': 8

In [94]:
import re
regex = re.compile('([e|i]+[n|s]+[t|f]+[j+p])')

def get_entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  #############################################################
  
  for tok in nlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]

get_entities(sentences[0])

['', 'ENFJ Personality Type']

In [95]:

entity_pairs = []

for i in sentences:
  entity_pairs.append(get_entities(i))
print(entity_pairs)

[['', 'ENFJ Personality Type'], ['', 'Extraverted  self'], ['', 'social  gatherings'], ['', ''], ['', ''], ['', 'Extraverts'], ['', 'limelight'], ['', 'people'], ['', 'principles'], ['', 'heart'], ['', 'ENFJs judge circumstances'], ['', 'others'], ['', 'harmony'], ['', 'Intuitive  future'], ['', 'possibilities'], ['', 'patterns'], ['', 'Intuitives'], ['', 'lines'], ['', 'deep  ideas'], ['', 'people'], ['', 'order'], ['', ''], ['', 'tasks'], ['', 'deadlines'], ['', ''], ['', ''], ['', 'day  day'], ['', '']]
